# Robustness Analysis: Multiplicative vs Additive Errors

This notebook compares the effectiveness of the toggling-frame robustness objective for both multiplicative and additive errors. It's strucuted as follows:
* Imports
* Problem Setups: Here we compare the effect of the toggling frame robustness objective on additive and multiplicative errors without any variational states.
    * Base case: unitary smooth pulse problem w/o any robustness or sentivity objectives
    * Adjoint: Optimize pulses for insenstivity to a) additive error, b) multiplcative error and c) both
    * Toggle Robustness: Optimize pulses for toggle-frame robustness to a - c above
    * Adjoint * Toggle: Both for a - c
* Plot Fidelity vs. Error
* Plot Trajectories and Control pulses
* Pareto Frontiers of Fidelity vs. Robustness Penality

## Imports

In [5]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();
Pkg.develop(path="../../QuantumCollocation.jl")
using PiccoloQuantumObjects
using QuantumCollocation
using ForwardDiff
using LinearAlgebra
# using Plots
using SparseArrays
using Statistics
using CairoMakie
using NamedTrajectories
using TrajectoryIndexingUtils
using Random

  Activating project at `~/Documents/research/pulses/project/notebooks/src`
   Resolving package versions...
  No Changes to `~/Documents/research/pulses/project/notebooks/src/Project.toml`
  No Changes to `~/Documents/research/pulses/project/notebooks/src/Manifest.toml`


## Problem Setups
Here, we first setup the quantum systems and solve for optimal pulses. There are ten problems to solve in total (the default smooth unitary pulse case, and for adjoint only, robustness only, adjoint + robustness, there are 3 cases (additive error, mujltiplcative error, both) giving nine problems).

#### Preliminary Variables

In [6]:
# Problem parameters
T = 50
Δt = 0.2
U_goal = GATES.X
H_drive = PAULIS.X
Hₑ = PAULIS.X
rob_scale = 1 / 8.0
a_bound = 0.2
dda_bound = 0.1
piccolo_opts = PiccoloOptions(verbose=false)
da_bound=Inf



Inf

#### Quanutm System Setup 

In [7]:
# Default Template
# H = aX
sys = QuantumSystem([H_drive])

# additive error template
# H = aX
# H_var = X
∂ₑHₐ = PAULIS.X

varsys_add = VariationalQuantumSystem(
    [H_drive],
    [∂ₑHₐ]
)

# Multiplicative error template
# H = aX
# H_drive_m = a -> a[1] * H_drive
# H_vars_array = Function[H_drive_m]

# varsys_mult = VariationalQuantumSystem(
#     H_drive_m,
#     H_vars_array,
#     1
# )

varsys_mult = VariationalQuantumSystem(
    a -> a[1] * PAULIS.X,
    Function[a -> a[1] * PAULIS.X],
    1
)

# Combine both errors template
# H = aX
# H_var = [aX + X, 0]

# H_drive_b = a -> a[1] * H_drive
# H_vars_array = Function[a -> a[1] * H_drive + PAULIS.X]

# varsys_both = VariationalQuantumSystem(
#     H_drive_b,
#     H_vars_array,
#     1
# )

varsys_both = VariationalQuantumSystem(
    a -> a[1] * PAULIS.X,
    Function[a -> a[1] * PAULIS.X, a -> PAULIS.X],
    1
)

VariationalQuantumSystem: levels = 2, n_drives = 1

In [8]:
println(varsys_add.n_drives)
println(varsys_mult.n_drives)
println(varsys_both.n_drives)


1
1
1


Sanity check before going brrrrrrrr

In [ ]:
default = UnitarySmoothPulseProblem(sys, U_goal, T, Δt; a_bound = a_bound, dda_bound = dda_bound)
solve!(default, max_iter=150, print_level=5)


In [ ]:
 add_prob = UnitaryVariationalProblem(
    varsys_add, U_goal, T, Δt;
    sensitive_times=[[T]],
    a_bound = a_bound,
    dda_bound = dda_bound,
    piccolo_options=piccolo_opts
)
solve!(add_prob, max_iter=200, print_level=5)

In [ ]:
Ũ⃗, Ũ⃗_vars = variational_unitary_rollout(add_prob.trajectory, varsys_add);
QuantumObjectives.unitary_fidelity_loss(Ũ⃗[:, end], GATES.X) |> println
QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[1][:, end ÷ 2]) |> println
# QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[2][:, end ÷ 2]) |> println
QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[1][:, end]) |> println
# QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[2][:, end]) |> println

In [ ]:
mult_prob = UnitaryVariationalProblem(
        varsys_mult, U_goal, T, Δt;
        sensitive_times=[[T]],
        a_bound = a_bound,
        dda_bound = dda_bound,
        piccolo_options=piccolo_opts
    )
solve!(mult_prob, max_iter=150, print_level=5)

In [ ]:
Ũ⃗, Ũ⃗_vars = variational_unitary_rollout(mult_prob.trajectory, varsys_mult);
QuantumObjectives.unitary_fidelity_loss(Ũ⃗[:, end], GATES.X) |> println
QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[1][:, end ÷ 2]) |> println
# QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[2][:, end ÷ 2]) |> println
QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[1][:, end]) |> println
# QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[2][:, end]) |> println

In [ ]:
both_prob = UnitaryVariationalProblem(
        varsys_both, U_goal, T, Δt;
        sensitive_times=[[T]],
        a_bound = a_bound,
        dda_bound = dda_bound,
        piccolo_options=piccolo_opts
    )
solve!(both_prob, max_iter=150, print_level=5)

In [ ]:
Ũ⃗, Ũ⃗_vars = variational_unitary_rollout(both_prob.trajectory, varsys_both);
QuantumObjectives.unitary_fidelity_loss(Ũ⃗[:, end], GATES.X) |> println
QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[1][:, end ÷ 2]) |> println
# QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[2][:, end ÷ 2]) |> println
QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[1][:, end]) |> println
# QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[2][:, end]) |> println

In [ ]:
H_drive_add = H_drive

f = Figure()
ax1 = Axis(f[1, 1], title="additive")
ax2 = Axis(f[2, 1], title="multiplicative")

colors = Makie.wong_colors()

εs = 0:0.01:.5

# default case (smooth, non-variational, w/o toggling obj)

ys = [unitary_rollout_fidelity(default.trajectory, QuantumSystem(ε * PAULIS.X, [PAULIS.X])) for ε in εs]
lines!(ax1, εs, ys, label="default", color=colors[1], linestyle=:solid)

ys = [unitary_rollout_fidelity(default.trajectory, QuantumSystem([(1 + ε) * PAULIS.X])) for ε in εs]
lines!(ax2, εs, ys, label="default", color=colors[1], linestyle=:solid)

# adjoint-only (no toggling obj)

# (mult)
ys = [unitary_rollout_fidelity(mult_prob.trajectory, QuantumSystem(ε * PAULIS.X, [PAULIS.X])) for ε in εs]
lines!(ax1, εs, ys, label="mult adjoint", color=colors[2], linestyle=:solid)

ys = [unitary_rollout_fidelity(mult_prob.trajectory, QuantumSystem([(1 + ε) * PAULIS.X])) for ε in εs]
lines!(ax2, εs, ys, label="mult adjoint", color=colors[2], linestyle=:solid)

# (add)
ys = [unitary_rollout_fidelity(add_prob.trajectory, QuantumSystem(ε * PAULIS.X, [PAULIS.X])) for ε in εs]
lines!(ax1, εs, ys, label="add adjoint", color=colors[3], linestyle=:solid)

ys = [unitary_rollout_fidelity(add_prob.trajectory, QuantumSystem([(1 + ε) * PAULIS.X])) for ε in εs]
lines!(ax2, εs, ys, label="add adjoint", color=colors[3], linestyle=:solid)

# (both)
# ys = [unitary_rollout_fidelity(both_proba.trajectory, QuantumSystem(ε * PAULIS.X, [PAULIS.X])) for ε in εs]
# lines!(ax1, εs, ys, label="both adjoint", color=colors[4], linestyle=:solid)

# ys = [unitary_rollout_fidelity(both_proba.trajectory, QuantumSystem([(1 + ε) * PAULIS.X])) for ε in εs]
# lines!(ax2, εs, ys, label="both adjoint", color=colors[4], linestyle=:solid)



Legend(f[1,2], ax1, position=:lb)
Legend(f[2,2], ax2, position=:lb)

f


In [ ]:
# default = UnitarySmoothPulseProblem(sys, U_goal, T, Δt; init_trajectory=traj, Q_t=1.0)
a_guess = [0.1*randn(1, T) for _ in 1:5]

default2 = UnitarySmoothPulseProblem(sys, U_goal, T, Δt; a_guess=deepcopy(a_guess[1]))
solve!(default2; max_iter=250, print_level=5)

p1 = CairoMakie.plot(default2.trajectory, [:a, :Ũ⃗])

display(p1)

Ũ⃗, Ũ⃗_vars = variational_unitary_rollout(default2.trajectory, varsys_add);
QuantumObjectives.unitary_fidelity_loss(Ũ⃗[:, end], GATES.X) |> println

#### Run Solvers

#### For adjoint w/o toggling frame

In [9]:
a_guess = [0.1*randn(1, T) for _ in 1:5]
a_bounds = fill(a_bound, 1)
da_bounds = fill(da_bound, 1)
dda_bounds = fill(dda_bound, 1)
control_bounds = (a_bounds, da_bounds, dda_bounds)
varsys_add_seeds = []
varsys_mult_seeds = []
varsys_both_seeds = []

n_drives = varsys_add.n_drives

for i in 1:5
    add_traj = initialize_trajectory(
                    U_goal,
                    T,
                    Δt,
                    n_drives,
                    control_bounds;
                    a_guess=deepcopy(a_guess[i]),
                    system=varsys_add
                )
    push!(varsys_add_seeds, add_traj)

    mult_traj = initialize_trajectory(
                U_goal,
                T,
                Δt,
                n_drives,
                control_bounds;
                a_guess=deepcopy(a_guess[i]),
                system=varsys_mult
            )
    push!(varsys_mult_seeds, mult_traj)

    both_traj = initialize_trajectory(
                U_goal,
                T,
                Δt,
                n_drives,
                control_bounds;
                a_guess=deepcopy(a_guess[i]),
                system=varsys_both
            )
    push!(varsys_both_seeds, both_traj)
end

In [ ]:
sweep_rob_loss_λ = [i for i in 2:6]
n_lambdas = length(sweep_rob_loss_λ)
n_seeds = length(a_guess)

In [ ]:
add_adj_probs_test = []

for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        
    # Add problem
    add_prob = UnitaryVariationalProblem(
        varsys_add, U_goal, T, Δt;
        init_trajectory=deepcopy(varsys_add_seeds[1]),
        # variational_scales=[rob_scale],
        sensitive_times=[[T]],
        piccolo_options=piccolo_opts,
        Q_s=10.0^(-λ)
    )
    solve!(add_prob; max_iter=200, print_level=5)
    push!(add_adj_probs_test, add_prob)
end

In [ ]:
for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
    println("\nProcessing λ = $λ")
    f = Figure()
    ax1 = Axis(f[1, 1], title="Additive noise, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")
    # ax2 = Axis(f[2, 1], title="Multiplicative noise, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")

    # Define system functions for additive and multiplicative noise
    additive_system = ε -> QuantumSystem(ε * PAULIS.X, [H_drive_add])
    multiplicative_system = ε -> QuantumSystem([(1 + ε) * H_drive])

    # Plot for both noise types
    # Additive noise
    ys_add = [unitary_rollout_fidelity(add_adj_probs_test[λ_idx].trajectory, additive_system(ε)) for ε in εs]
    lines!(ax1, εs, ys_add, label="add, λ=$λ", color=colors[1], linestyle=:solid)
    
    # Multiplicative noise
    # ys_mult = [unitary_rollout_fidelity(add_adj_probs[λ_idx].trajectory, multiplicative_system(ε)) for ε in εs]
    # lines!(ax2, εs, ys_mult, label="mult, λ=$λ, seed_idx=$seed_idx", color=colors[1], linestyle=:solid)


    # Add legends
    Legend(f[1, 2], ax1, position=:lb)
    # Legend(f[2, 2], ax2, position=:lb)
    
    display(f)
end

In [ ]:
for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
    Ũ⃗, Ũ⃗_vars = variational_unitary_rollout(add_adj_probs_test[λ_idx].trajectory, varsys_add);
    QuantumObjectives.unitary_fidelity_loss(Ũ⃗[:, end], GATES.X) |> println
    QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[1][:, end ÷ 2]) |> println
    # QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[2][:, end ÷ 2]) |> println
    QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[1][:, end]) |> println
    # QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[2][:, end]) |> println
end

In [ ]:
add_adj_probs = Matrix{Any}(undef, n_seeds, n_lambdas)

for i in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        
        # Add problem
        add_prob = UnitaryVariationalProblem(
            varsys_add, U_goal, T, Δt;
            init_trajectory=deepcopy(varsys_add_seeds[i]),
            variational_scales=[rob_scale],
            sensitive_times=[[T]],
            piccolo_options=piccolo_opts,
            Q_s=λ
        )
        solve!(add_prob; max_iter=200, print_level=5)
        add_adj_probs[i, λ_idx] = add_prob
    end
end

In [ ]:
H_drive_add = H_drive
εs = 0:0.01:0.5
colors = Makie.wong_colors()
# for seed_idx in 1:n_seeds
#     for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
#         for ε in εs
#             println("ε=$ε, λ=$λ, seed num = $seed_idx")
#             Ũ⃗, Ũ⃗_vars = variational_unitary_rollout(add_adj_probs[seed_idx, λ_idx].trajectory, varsys_add);
#             # QuantumObjectives.unitary_fidelity_loss(Ũ⃗_vars[1][:, end]) |> println
#             unitary_rollout_fidelity(add_adj_probs[seed_idx, λ_idx].trajectory, varsys_add) |> println
#             display(p2)
#         end
#     end
# end

for seed_idx in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        println("\nProcessing λ = $λ")
        f = Figure()
        ax1 = Axis(f[1, 1], title="Additive noise, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")
        ax2 = Axis(f[2, 1], title="Multiplicative noise, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")

        # Define system functions for additive and multiplicative noise
        additive_system = ε -> QuantumSystem(ε * PAULIS.X, [H_drive_add])
        multiplicative_system = ε -> QuantumSystem([(1 + ε) * H_drive])

        # Plot for both noise types
        # Additive noise
        ys_add = [unitary_rollout_fidelity(add_adj_probs[seed_idx, λ_idx].trajectory, additive_system(ε)) for ε in εs]
        lines!(ax1, εs, ys_add, label="add, λ=$λ, seed_idx=$seed_idx", color=colors[1], linestyle=:solid)
        
        # Multiplicative noise
        ys_mult = [unitary_rollout_fidelity(add_adj_probs[seed_idx, λ_idx].trajectory, multiplicative_system(ε)) for ε in εs]
        lines!(ax2, εs, ys_mult, label="mult, λ=$λ, seed_idx=$seed_idx", color=colors[1], linestyle=:solid)


        # Add legends
        Legend(f[1, 2], ax1, position=:lb)
        Legend(f[2, 2], ax2, position=:lb)
        
        display(f)
    end
end


In [ ]:
H_drive_add = H_drive
εs = 0:0.01:0.5
colors = Makie.wong_colors()

# # Function to compute average fidelity across seeds
# function compute_avg_fidelity(probs_matrix, λ_idx, system_func, ε)
#     fidelities = Float64[]
#     for seed_idx in 1:n_seeds
#         prob = probs_matrix[seed_idx, λ_idx]
#         fid = unitary_rollout_fidelity(prob.trajectory, system_func(ε))
#         push!(fidelities, fid)
#     end
#     return mean(fidelities)
# end

# Plot for each λ
for seed_idx in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        println("\nProcessing λ = $λ")
        f = Figure()
        ax1 = Axis(f[1, 1], title="Additive noise, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")
        ax2 = Axis(f[2, 1], title="Multiplicative noise, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")

        # Define system functions for additive and multiplicative noise
        additive_system = ε -> QuantumSystem(ε * PAULIS.X, [H_drive_add])
        multiplicative_system = ε -> QuantumSystem([(1 + ε) * H_drive])

        # Plot data structure: (matrix, label, color)
        plot_configs = [
            # (default_probs, "default", colors[1]),
            # (mult_adj_probs, "mult adjoint", colors[2]),
            (add_adj_probs, "add adjoint", colors[3])
            # (both_adj_probs, "both adjoint", colors[4])
        ]

        # Plot for both noise types
        # for (probs_matrix, label, color) in plot_configs
            # Additive noise
        # prob = [probs_matrix[seed_idx, λ_idx] for ε in εs]
        ys_add = [unitary_rollout_fidelity(add_adj_probs[seed_idx, λ_idx].trajectory, QuantumSystem(ε * PAULIS.X, [H_drive_add])) for ε in εs]
        lines!(ax1, εs, ys_add, label="add adjoint", color=colors[3], linestyle=:solid)
        
        # Multiplicative noise
        # prob = [probs_matrix[seed_idx, λ_idx] for ε in εs]
        ys_mult = [unitary_rollout_fidelity(add_adj_probs[seed_idx, λ_idx].trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
        lines!(ax2, εs, ys_mult, label="mult adjoint", color=colors[3], linestyle=:solid)
        # end

        # Add legends
        Legend(f[1, 2], ax1, position=:lb)
        Legend(f[2, 2], ax2, position=:lb)
        
        display(f)
    end
end


In [ ]:
mult_adj_probs = Matrix{Any}(undef, n_seeds, n_lambdas)
for i in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        # Mult problem
        mult_prob = UnitaryVariationalProblem(
            varsys_mult, U_goal, T, Δt;
            init_trajectory=deepcopy(varsys_mult_seeds[i]),
            variational_scales=[rob_scale],
            sensitive_times=[[T]], 
            piccolo_options=piccolo_opts,
            Q_s=λ
        )
        solve!(mult_prob; max_iter=200, print_level=5)
        mult_adj_probs[i, λ_idx] = mult_prob
    end
end

In [ ]:
both_adj_probs = Matrix{Any}(undef, n_seeds, n_lambdas)

# Optimization loop with index counter
for i in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        


        # Both problem
        both_prob = UnitaryVariationalProblem(
            varsys_both, U_goal, T, Δt;
            init_trajectory=deepcopy(varsys_both_seeds[i]),
            variational_scales=[rob_scale,rob_scale],
            sensitive_times=[[T]],
            piccolo_options=piccolo_opts,
            Q_s=λ
        )
        solve!(both_prob; max_iter=200, print_level=5)
        both_adj_probs[i, λ_idx] = both_prob
    end
end

In [ ]:
H_drive_add = H_drive
εs = 0:0.01:0.5
colors = Makie.wong_colors()

# # Function to compute average fidelity across seeds
# function compute_avg_fidelity(probs_matrix, λ_idx, system_func, ε)
#     fidelities = Float64[]
#     for seed_idx in 1:n_seeds
#         prob = probs_matrix[seed_idx, λ_idx]
#         fid = unitary_rollout_fidelity(prob.trajectory, system_func(ε))
#         push!(fidelities, fid)
#     end
#     return mean(fidelities)
# end

# Plot for each λ
for seed_idx in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        println("\nProcessing λ = $λ")
        f = Figure()
        ax1 = Axis(f[1, 1], title="Additive noise, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")
        ax2 = Axis(f[2, 1], title="Multiplicative noise, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")

        # Define system functions for additive and multiplicative noise
        additive_system = ε -> QuantumSystem(ε * PAULIS.X, [H_drive_add])
        multiplicative_system = ε -> QuantumSystem([(1 + ε) * H_drive])

        # Plot data structure: (matrix, label, color)
        plot_configs = [
            # (default_probs, "default", colors[1]),
            # (mult_adj_probs, "mult adjoint", colors[2]),
            (add_adj_probs, "add adjoint", colors[3])
            # (both_adj_probs, "both adjoint", colors[4])
        ]

        # Plot for both noise types
        # for (probs_matrix, label, color) in plot_configs
            # Additive noise
        # prob = [probs_matrix[seed_idx, λ_idx] for ε in εs]
        ys_add = [unitary_rollout_fidelity(add_adj_probs[seed_idx, λ_idx].trajectory, QuantumSystem(ε * PAULIS.X, [H_drive_add])) for ε in εs]
        lines!(ax1, εs, ys_add, label="add adjoint", color=colors[3], linestyle=:solid)
        
        # Multiplicative noise
        # prob = [probs_matrix[seed_idx, λ_idx] for ε in εs]
        ys_mult = [unitary_rollout_fidelity(add_adj_probs[seed_idx, λ_idx].trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
        lines!(ax2, εs, ys_mult, label="mult adjoint", color=colors[3], linestyle=:solid)
        # end

        # Add legends
        Legend(f[1, 2], ax1, position=:lb)
        Legend(f[2, 2], ax2, position=:lb)
        
        display(f)
    end
end


In [ ]:
default_probs = Matrix{Any}(undef, n_seeds, n_lambdas)

for i in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        defaults = UnitarySmoothPulseProblem(sys, U_goal, T, Δt; a_guess=deepcopy(a_guess[i]))
        solve!(default; max_iter=250, print_level=5)
        default_probs[i,λ_idx] = defaults
    end
end

In [ ]:
H_drive_add = H_drive
εs = 0:0.01:0.5
colors = Makie.wong_colors()

# Function to compute average fidelity across seeds
function compute_avg_fidelity(probs_matrix, λ_idx, system_func, ε)
    fidelities = Float64[]
    for seed_idx in 1:n_seeds
        prob = probs_matrix[seed_idx, λ_idx]
        fid = unitary_rollout_fidelity(prob.trajectory, system_func(ε))
        push!(fidelities, fid)
    end
    return mean(fidelities)
end

# Plot for each λ
for seed_idx in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        println("\nProcessing λ = $λ")
        f = Figure()
        ax1 = Axis(f[1, 1], title="Additive noise, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")
        ax2 = Axis(f[2, 1], title="Multiplicative noise, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")

        # Define system functions for additive and multiplicative noise
        additive_system = ε -> QuantumSystem(ε * PAULIS.X, [H_drive_add])
        multiplicative_system = ε -> QuantumSystem([(1 + ε) * H_drive])

        # Plot data structure: (matrix, label, color)
        plot_configs = [
            (default_probs, "default", colors[1]),
            (mult_adj_probs, "mult adjoint", colors[2]),
            (add_adj_probs, "add adjoint", colors[3]),
            (both_adj_probs, "both adjoint", colors[4])
        ]

        # Plot for both noise types
        for (probs_matrix, label, color) in plot_configs
            # Additive noise
            prob = [probs_matrix[seed_idx, λ_idx]) for ε in εs]
            ys_add = [unitary_rollout_fidelity(prob.trajectory, additive_system) for ε in εs]
            lines!(ax1, εs, ys_add, label=label, color=color, linestyle=:solid)
            
            # Multiplicative noise
            prob = [probs_matrix[seed_idx, λ_idx]) for ε in εs]
            ys_mult = [unitary_rollout_fidelity(prob.trajectory, multiplicative_system) for ε in εs]
            lines!(ax2, εs, ys_mult, label=label, color=color, linestyle=:solid)
        end

        # Add legends
        Legend(f[1, 2], ax1, position=:lb)
        Legend(f[2, 2], ax2, position=:lb)
        
        display(f)
    end
end


In [ ]:
H_drive_add = H_drive

for i in length(sweep_rob_loss_λ)
    println("\nProcessing λ = $(sweep_rob_loss_λ[i])")
    f = Figure()
    ax1 = Axis(f[1, 1], title="additive, λ = $(sweep_rob_loss_λ[i])")
    ax2 = Axis(f[2, 1], title="multiplicative, λ = $(sweep_rob_loss_λ[i])")

    colors = Makie.wong_colors()

    εs = 0:0.01:.5

    # default case (smooth, non-variational, w/o toggling obj)
    default = defaults[i]
    ys = [unitary_rollout_fidelity(default.trajectory, QuantumSystem(ε * (PAULIS.X), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="default", color=colors[1], linestyle=:solid)

    ys = [unitary_rollout_fidelity(default.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="default", color=colors[1], linestyle=:solid)

    # adjoint-only (no toggling obj)

    # (mult)
    mult_adj_prob = mult_adj_probs[i]
    ys = [unitary_rollout_fidelity(mult_adj_prob.trajectory, QuantumSystem(ε * (PAULIS.X), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="mult adjoint", color=colors[2], linestyle=:solid)

    ys = [unitary_rollout_fidelity(mult_adj_prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="mult adjoint", color=colors[2], linestyle=:solid)

    # (add)
    add_adj_prob = add_adj_probs[i]
    ys = [unitary_rollout_fidelity(add_adj_prob.trajectory, QuantumSystem(ε * (PAULIS.X), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="add adjoint", color=colors[3], linestyle=:solid)

    ys = [unitary_rollout_fidelity(add_adj_prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="add adjoint", color=colors[3], linestyle=:solid)
    
    # (both)
    both_adj_prob = both_adj_probs[i]
    ys = [unitary_rollout_fidelity(both_adj_prob.trajectory, QuantumSystem(ε * (PAULIS.X), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="both adjoint", color=colors[4], linestyle=:solid)

    ys = [unitary_rollout_fidelity(both_adj_prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="both adjoint", color=colors[4], linestyle=:solid)

    Legend(f[1,2], ax1, position=:lb)
    Legend(f[2,2], ax2, position=:lb)
    display(f)
end

#### For toggling frame w/o adjoint robustness

example problem

In [19]:
ex_tog_default = UnitarySmoothPulseProblem(sys, U_goal, T, Δt; activate_rob_loss=true, H_err=H_drive, Q_t=1.0, a_guess=deepcopy(a_guess[1]))
solve!(ex_tog_default, max_iter=150, print_level=5)

ex_add_prob = UnitaryVariationalProblem(
            varsys_add, U_goal, T, Δt;
            init_trajectory=deepcopy(varsys_add_seeds[1]),
            variational_scales=[rob_scale],
            sensitive_times=[[T]],
            piccolo_options=piccolo_opts,
            activate_rob_loss=true,
            H_err=H_drive,
            Q_s=0.0,
            Q_r=0.0,
            Q_t=1.0
        )
solve!(ex_add_prob; max_iter=150, print_level=5)

ex_mult_prob = UnitaryVariationalProblem(
    varsys_mult, U_goal, T, Δt;
    init_trajectory=deepcopy(varsys_mult_seeds[1]),
    variational_scales=[rob_scale],
    sensitive_times=[[T]], 
    piccolo_options=piccolo_opts,
    activate_rob_loss=true,
    H_err=H_drive,
    Q_s=0.0,
    Q_r=0.0,
    Q_t=1.0
)
solve!(ex_add_prob; max_iter=150, print_level=5)

ex_both_prob = UnitaryVariationalProblem(
            varsys_both, U_goal, T, Δt;
            init_trajectory=deepcopy(varsys_both_seeds[1]),
            variational_scales=[rob_scale,rob_scale],
            sensitive_times=[[T]],
            piccolo_options=piccolo_opts,
            activate_rob_loss=true,
            H_err=H_drive,
            Q_s=0.0,
            Q_r=0.0,
            Q_t=1.0
        )
solve!(ex_add_prob; max_iter=150, print_level=5)

In [ ]:
p1 = CairoMakie.plot(ex_tog_default.trajectory, [:a, :Ũ⃗])
p2 = CairoMakie.plot(ex_add_prob.trajectory, [:a, :Ũ⃗])
p3 = CairoMakie.plot(ex_mult_prob.trajectory, [:a, :Ũ⃗])
p4 = CairoMakie.plot(ex_both_prob.trajectory, [:a, :Ũ⃗])

display(p1)
display(p2)
display(p3)
display(p4)

In [21]:
Ũ⃗, Ũ⃗_vars = variational_unitary_rollout(ex_tog_default.trajectory, varsys_add)
println(QuantumObjectives.unitary_fidelity_loss(Ũ⃗[:, end], GATES.X))


In [22]:
Ũ⃗_t, Ũ⃗_vars = variational_unitary_rollout(ex_add_prob.trajectory, varsys_add)
QuantumObjectives.unitary_fidelity_loss(Ũ⃗_t[:, end], GATES.X) |> println


In [23]:
# default case

# additive

# mult
Ũ⃗, Ũ⃗_vars = variational_unitary_rollout(ex_mult_prob.trajectory, varsys_mult)
QuantumObjectives.unitary_fidelity_loss(Ũ⃗[:, end], GATES.X) |> println


# both problem
Ũ⃗, Ũ⃗_vars = variational_unitary_rollout(ex_both_prob.trajectory, varsys_add)
QuantumObjectives.unitary_fidelity_loss(Ũ⃗[:, end], GATES.X) |> println

Ũ⃗, Ũ⃗_vars = variational_unitary_rollout(ex_both_prob.trajectory, varsys_mult);
QuantumObjectives.unitary_fidelity_loss(Ũ⃗[:, end], GATES.X) |> println

Ũ⃗, Ũ⃗_vars = variational_unitary_rollout(ex_both_prob.trajectory, varsys_both);
QuantumObjectives.unitary_fidelity_loss(Ũ⃗[:, end], GATES.X) |> println

In [ ]:
# Setup optimization parameters
sweep_rob_loss_λ = [i for i in 0.0:5.0]
n_lambdas = length(sweep_rob_loss_λ)
n_seeds = length(a_guess)

# Initialize result storage
tog_defaults = Matrix{Any}(undef, n_seeds, n_lambdas)
add_tog_probs = Matrix{Any}(undef, n_seeds, n_lambdas)
mult_tog_probs = Matrix{Any}(undef, n_seeds, n_lambdas)
both_tog_probs = Matrix{Any}(undef, n_seeds, n_lambdas)

# Optimization loop with index counter
for i in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)

        tog_default = UnitarySmoothPulseProblem(sys, U_goal, T, Δt; activate_rob_loss=true, H_err=H_drive, Q_t=λ)
        solve!(tog_default, max_iter=150, print_level=5)
        tog_defaults[i, λ_idx] = tog_default
            
        # Add problem
        add_prob = UnitaryVariationalProblem(
            varsys_add, U_goal, T, Δt;
            init_trajectory=deepcopy(varsys_add_seeds[i].trajectory),
            piccolo_options=piccolo_opts,
            activate_rob_loss=true,
            H_err=H_drive,
            Q_s=0.0,
            Q_r=0.0,
            Q_t=λ
        )
        solve!(add_prob; max_iter=150, print_level=5)
        add_tog_probs[i, λ_idx] = add_prob

        # Mult problem
        mult_prob = UnitaryVariationalProblem(
            varsys_mult, U_goal, T, Δt;
            init_trajectory=deepcopy(varsys_mult_seeds[i]),
            piccolo_options=piccolo_opts,
            activate_rob_loss=true,
            H_err=H_drive,
            Q_s=0.0,
            Q_r=0.0,
            Q_t=λ
        )
        solve!(mult_prob; max_iter=150, print_level=5)
        mult_tog_probs[i, λ_idx] = mult_prob

        # Both problem
        both_prob = UnitaryVariationalProblem(
            varsys_both, U_goal, T, Δt;
            init_trajectory=deepcopy(varsys_both_seeds[i]),
            piccolo_options=piccolo_opts,
            activate_rob_loss=true,
            H_err=H_drive,
            Q_s=0.0,
            Q_r=0.0,
            Q_t=λ
        )
        solve!(both_prob; max_iter=150, print_level=5)
        both_tog_probs[i, λ_idx] = both_prob
    end
end



# rob_defaults = []
# add_rob_probs = []
# mult_rob_probs = []
# both_rob_probs = []
# sweep_rob_loss_λ = [i for i in 0.0:0.2:1.0]
# for λ in sweep_rob_loss_λ
#     rob_default = UnitarySmoothPulseProblem(sys, U_goal, T, Δt; activate_rob_loss=true, H_err=H_drive, Q_t=λ)
#     solve!(rob_default, max_iter=150, print_level=5)
#     push!(rob_defaults, rob_default)
    
#     add_rob_prob = UnitaryVariationalProblem(
#             varsys_add, U_goal, T, Δt;
#             variational_scales=[rob_scale],
#             sensitive_times=[[T]],
#             activate_rob_loss=true,
#             H_err=H_drive,
#             Q_s=0.0,
#             Q_r=0.0,
#             Q_t=λ,
#             piccolo_options=piccolo_opts
#         )
#     solve!(add_rob_prob, max_iter=100, print_level=5)
#     push!(add_rob_probs, add_rob_prob)

#     mult_rob_prob = UnitaryVariationalProblem(
#             varsys_mult, U_goal, T, Δt;
#             variational_scales=[rob_scale],
#             sensitive_times=[[T]],
#             activate_rob_loss=true,
#             H_err=H_drive,
#             Q_s=0.0,
#             Q_r=0.0,
#             Q_t=λ,
#             piccolo_options=piccolo_opts
#         )
#     solve!(mult_rob_prob, max_iter=100, print_level=5)
#     push!(mult_rob_probs, mult_rob_prob)

#     both_rob_prob = UnitaryVariationalProblem(
#             varsys_both, U_goal, T, Δt;
#             variational_scales=[rob_scale],
#             sensitive_times=[[T]],
#             activate_rob_loss=true,
#             H_err=H_drive,
#             Q_s=0.0,
#             Q_r=0.0,
#             Q_t=λ,
#             piccolo_options=piccolo_opts
#         )
#     solve!(both_rob_prob, max_iter=100, print_level=5)
#     push!(both_rob_probs, both_rob_prob)
# end

    constructing UnitarySmoothPulseProblem...
	using integrator: typeof(UnitaryIntegrator)
	control derivative names: [:da, :dda]
	applying timesteps_all_equal constraint: Δt
    initializing optimizer...
        applying constraint: timesteps all equal constraint
        applying constraint: initial value of Ũ⃗
        applying constraint: initial value of a
        applying constraint: final value of a
        applying constraint: bounds on a
        applying constraint: bounds on da
        applying constraint: bounds on dda
        applying constraint: bounds on Δt

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, runnin

Excessive output truncated after 524349 bytes.

## Plot Fidelity vs. Errors

In [ ]:
H_drive_add = H_drive
εs = 0:0.01:0.5
colors = Makie.wong_colors()

# # Function to compute average fidelity across seeds
# function compute_avg_fidelity(probs_matrix, λ_idx, system_func, ε)
#     fidelities = Float64[]
#     for seed_idx in 1:n_seeds
#         prob = probs_matrix[seed_idx, λ_idx]
#         fid = unitary_rollout_fidelity(prob.trajectory, system_func(ε))
#         push!(fidelities, fid)
#     end
#     return mean(fidelities)
# end

# Plot for each λ
for seed_idx in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
        println("\nProcessing λ = $λ")
        f = Figure()
        ax1 = Axis(f[1, 1], title="Additive noise, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")
        ax2 = Axis(f[2, 1], title="Multiplicative noise, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")
        ax3 = Axis(f[2, 1], title="Both noises, λ = $λ", xlabel="Error (ε)", ylabel="Average Fidelity")

        # Define system functions for additive and multiplicative noise
        additive_system = ε -> QuantumSystem(ε * PAULIS.X, [H_drive_add])
        multiplicative_system = ε -> QuantumSystem([(1 + ε) * H_drive])
        both_system = ε -> QuantumSystem(ε * PAULIS.X, [(1 + ε) * H_drive])
        # Plot data structure: (matrix, label, color)
        plot_configs = [
            (tog_defaults, "default", colors[1]),
            (add_tog_probs, "mult toggle rob", colors[2]),
            (mult_tog_probs, "add toggle rob", colors[3])
            (both_tog_probs, "both toggle rob", colors[4])
        ]

        # Plot for both noise types
        for (probs_matrix, label, color) in plot_configs
            # Additive noise
            prob = probs_matrix[seed_idx, λ_idx]
            ys_add = [unitary_rollout_fidelity(prob.trajectory, QuantumSystem(ε * PAULIS.X, [H_drive_add])) for ε in εs]
            lines!(ax1, εs, ys_add, label=label, color=color, linestyle=:solid)
            
            # Multiplicative noise
            ys_mult = [unitary_rollout_fidelity(prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
            lines!(ax2, εs, ys_mult, label=label, color=color, linestyle=:solid)

            ys_both = [unitary_rollout_fidelity(prob.trajectory, QuantumSystem(ε * PAULIS.X, [(1 + ε) * H_drive])) for ε in εs]
            lines!(ax3, εs, ys_both, label=label, color=color, linestyle=:solid)
        end

        # Add legends
        Legend(f[1, 2], ax1, position=:lb)
        Legend(f[2, 2], ax2, position=:lb)
        Legend(f[2, 2], ax3, position=:lb)

        display(f)
    end
end


In [ ]:
H_drive_add = H_drive

for i in length(sweep_rob_loss_λ)
    println("\nProcessing λ = $(sweep_rob_loss_λ[i])")
    f = Figure()
    ax1 = Axis(f[1, 1], title="additive, λ = $(sweep_rob_loss_λ[i])")
    ax2 = Axis(f[2, 1], title="multiplicative, λ = $(sweep_rob_loss_λ[i])")

    colors = Makie.wong_colors()

    εs = 0:0.01:.5

    # default case (smooth, non-variational, w/o toggling obj)
    default = defaults[i]
    ys = [unitary_rollout_fidelity(default.trajectory, QuantumSystem(ε * (PAULIS.X), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="default", color=colors[1], linestyle=:solid)

    ys = [unitary_rollout_fidelity(default.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="default", color=colors[1], linestyle=:solid)

    # adjoint-only (no toggling obj)

    # (mult)
    mult_adj_prob = mult_adj_probs[i]
    ys = [unitary_rollout_fidelity(mult_adj_prob.trajectory, QuantumSystem(ε * (PAULIS.X), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="mult adjoint", color=colors[2], linestyle=:solid)

    ys = [unitary_rollout_fidelity(mult_adj_prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="mult adjoint", color=colors[2], linestyle=:solid)

    # (add)
    add_adj_prob = add_adj_probs[i]
    ys = [unitary_rollout_fidelity(add_adj_prob.trajectory, QuantumSystem(ε * (PAULIS.X), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="add adjoint", color=colors[3], linestyle=:solid)

    ys = [unitary_rollout_fidelity(add_adj_prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="add adjoint", color=colors[3], linestyle=:solid)
    
    # (both)
    both_adj_prob = both_adj_probs[i]
    ys = [unitary_rollout_fidelity(both_adj_prob.trajectory, QuantumSystem(ε * (PAULIS.X), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="both adjoint", color=colors[4], linestyle=:solid)

    ys = [unitary_rollout_fidelity(both_adj_prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="both adjoint", color=colors[4], linestyle=:solid)

    Legend(f[1,2], ax1, position=:lb)
    Legend(f[2,2], ax2, position=:lb)
    display(f)
end

In [ ]:
H_drive_add = H_drive

for i in length(sweep_rob_loss_λ)
    println("\nProcessing λ = $(sweep_rob_loss_λ[i])")
    f = Figure()
    ax1 = Axis(f[1, 1], title="additive, λ = $(sweep_rob_loss_λ[i])")
    ax2 = Axis(f[2, 1], title="multiplicative, λ = $(sweep_rob_loss_λ[i])")

    colors = Makie.wong_colors()

    εs = 0:0.01:.5

    # default case (smooth, non-variational, w/o toggling obj)
    # default = defaults[i]
    # ys = [unitary_rollout_fidelity(default.trajectory, QuantumSystem(ε * (H_drive), [H_drive_add])) for ε in εs]
    # lines!(ax1, εs, ys, label="default", color=colors[1], linestyle=:solid)

    # ys = [unitary_rollout_fidelity(default.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    # lines!(ax2, εs, ys, label="default", color=colors[1], linestyle=:solid)

    # toggling-obj-only (no variational states)

    rob_default = rob_defaults[i]
    ys = [unitary_rollout_fidelity(rob_default.trajectory, QuantumSystem(ε * (PAULIS.X), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="non-variational robust", color=colors[2], linestyle=:solid)

    ys = [unitary_rollout_fidelity(rob_default.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="non-variational robust", color=colors[2], linestyle=:solid)

    # Robust+adjoint (addtive)
    add_rob_prob = add_rob_probs[i]
    ys = [unitary_rollout_fidelity(add_rob_prob.trajectory, QuantumSystem(ε * (PAULIS.X), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="Robust+adjoint (add)", color=colors[3], linestyle=:solid)

    ys = [unitary_rollout_fidelity(add_rob_prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="Robust+adjoint (add)", color=colors[3], linestyle=:solid)

    # Robust+adjoint (mult)
    mult_rob_prob = mult_rob_probs[i]
    ys = [unitary_rollout_fidelity(mult_rob_prob.trajectory, QuantumSystem(ε * (PAULIS.X), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="Robust+adjoint (mult)", color=colors[4], linestyle=:solid)

    ys = [unitary_rollout_fidelity(mult_rob_prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="Robust+adjoint (mult)", color=colors[4], linestyle=:solid)

    # Robust+adjoint (both)
    both_rob_prob = both_rob_probs[i]
    ys = [unitary_rollout_fidelity(both_rob_prob.trajectory, QuantumSystem(ε * (PAULIS.X), [H_drive_add])) for ε in εs]
    lines!(ax1, εs, ys, label="Robust+adjoint (both)", color=colors[5], linestyle=:dash)

    ys = [unitary_rollout_fidelity(both_rob_prob.trajectory, QuantumSystem([(1 + ε) * H_drive])) for ε in εs]
    lines!(ax2, εs, ys, label="Robust+adjoint (both)", color=colors[5], linestyle=:dash)


    Legend(f[1,2], ax1, position=:lb)
    Legend(f[2,2], ax2, position=:lb)
    
    display(f)
end
    

#TODO: add plots of trajectories

## Pareto Frontier Plots

### Robust (w/o adjoint)

In [ ]:
# Initialize storage for fidelities for each lambda
εs = 0:0.01:.5
error_magnitudes = [x for x in εs]
n_lambdas = length(sweep_rob_loss_λ)
n_errors = length(error_magnitudes)
n_seeds = length(a_guess)
templates = [tog_defaults, add_tog_probs, mult_tog_probs, both_tog_probs]
# Store fidelities for each lambda and error combination
base_fidelities= zeros(n_seeds, n_lambdas)
additive_fidelities = zeros(n_seeds, n_lambdas)
multiplicative_fidelities = zeros(n_seeds, n_lambdas)
obj_vals = zeros(n_seeds, n_lambdas)

# Function to compute average fidelity across seeds
# function compute_avg_fidelity(prob, λ_idx, system_func, error_magnitudes)
#     fidelities = Float64[]
#     for ε in 1:error_magnitudes
#         fid = unitary_rollout_fidelity(prob.trajectory, system_func(ε))
#         push!(fidelities, fid)
#     end
#     return mean(fidelities)
err_sys = QuantumSystem([H_drive_add])
endadditive_system = ε -> QuantumSystem(ε * PAULIS.X, [H_drive_add])
multiplicative_system = ε -> QuantumSystem([(1 + ε) * H_drive])
both_system = ε -> QuantumSystem(ε * PAULIS.X, [(1 + ε) * H_drive])

for template in templates
    # Calculate fidelities for each problem (lambda value)
    for seed_idx in 1:n_seeds
        # Calculate fidelities for each error magnitude
        for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
            prob = template[seed_idx,λ_idx]
            println("\nProcessing λ = $λ")
            obj = QuantumObjectives.FirstOrderObjective(Hₑ, prob.trajectory, [T]; Q_t=1.0)#prob.trajectory.Ũ⃗ᵥ1[:, end]
            Z_vec = vec(prob.trajectory)
            obj_val = obj.L(Z_vec)
            obj_vals[seed_idx, λ_idx] = obj_val
            
            # Base case
            base_fidelity = unitary_rollout_fidelity(prob.trajectory, err_sys)
            base_fidelities[seed_idx, λ_idx] = base_fidelity
                        
            # Additive case
            # add_err_sys = QuantumSystem(ε * (H_drive), [H_drive_add])
            add_fidelity = mean([unitary_rollout_fidelity(prob.trajectory, additive_system(ε)) for ε in 0.0:0.05:0.5])
            additive_fidelities[seed_idx, λ_idx] = add_fidelity
            
            # Multiplicative case
            mult_err_sys = QuantumSystem([(1 + ε) * H_drive])
            mult_fidelity = mean([unitary_rollout_fidelity(prob.trajectory, multiplicative_system(ε)) for ε in 0.0:0.05:0.5])
            multiplicative_fidelities[seed_idx, λ_idx] = mult_fidelity
        end
    end

    
    p1 = Plots.plot(xlabel="Robustness ≡ |tr(R'R)|, where R = ∑ₜ(uₜ' Hₑ uₜ)", 
            ylabel="Fidelity",
            title="base Error: Fidelity vs Robustness (unitless)",
            legendfontsize=10, titlefontsize=12,
            grid=true, gridwidth=1, gridcolor=:gray, gridalpha=0.3,
            legend=:topright)

    for seed_idx in 1:n_seeds
        # Calculate fidelities for each error magnitude
        for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
            Plots.plot!(p1, obj_vals[seed_idx, λ_idx], base_fidelities[seed_idx, λ_idx],
                marker=:circle, markersize=3, linewidth=2)


    # Plot 2: Additive Error - Fidelity vs Robustness for different error magnitudes
    p2 = Plots.plot(xlabel="Robustness ≡ |tr(R'R)|, where R = ∑ₜ(uₜ' Hₑ uₜ)", 
            ylabel="Fidelity",
            title="Additive Error: Fidelity vs Robustness (unitless)",
            legendfontsize=10, titlefontsize=12,
            grid=true, gridwidth=1, gridcolor=:gray, gridalpha=0.3,
            legend=:topright)

    for seed_idx in 1:n_seeds
        # Calculate fidelities for each error magnitude
        for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
            Plots.plot!(p2, obj_vals[seed_idx, λ_idx], additive_fidelities[seed_idx, λ_idx],
                marker=:circle, markersize=3, linewidth=2)

    # Plot 3: Multiplicative Error - Fidelity vs Robustness for different error magnitudes
    p3 = Plots.plot(xlabel="Robustness ≡ |tr(R'R)|, where R = ∑ₜ(uₜ' Hₑ uₜ)", 
            ylabel="Fidelity",
            title="Multiplicative Error: Fidelity vs Robustness (unitless)",
            legendfontsize=10, titlefontsize=12,
            grid=true, gridwidth=1, gridcolor=:gray, gridalpha=0.3,
            legend=:topright)
    for seed_idx in 1:n_seeds
        # Calculate fidelities for each error magnitude
        for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
            Plots.plot!(p3, obj_vals[seed_idx, λ_idx], multiplicative_fidelities[seed_idx, λ_idx], 
                marker=:square, markersize=3, linewidth=2)


    # Display all plots
    display(p1)
    display(p2)
    display(p3)

    # Detailed results table
    println("\n=== Detailed Results Table ===")
    for seed_idx in 1:n_seeds
        println("\nseed idx = $seed_idx:")
        println("Weight λ  | Add Fidelity | Mult Fidelity")
        println("-" ^ 40)
        for (λ_idx, λ) in enumerate(sweep_rob_loss_λ)
            println("$(lpad(round(λ, digits=4), 7)) | $(lpad(round(additive_fidelities[seed_idx, λ_idx], digits=6), 12)) | $(lpad(round(multiplicative_fidelities[seed_idx, λ_idx], digits=6), 13))")
        end
    end
end

### additive robust prob